In [1]:
import tensorflow as tf
import random
import numpy as np
import math
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
import optuna
from optuna.storages import JournalStorage, JournalFileStorage
from sklearn.model_selection import StratifiedKFold
from scikeras.wrappers import KerasClassifier
import numpy as np
import pandas as pd
import sklearn
from matplotlib import pyplot as plt
from sklearn.base import BaseEstimator, ClassifierMixin, clone
from sklearn.metrics import (
    fbeta_score,
    accuracy_score,
)
from sklearn.model_selection import (
    train_test_split,
)
from sklearn.utils.multiclass import unique_labels
from sklearn.utils.validation import check_is_fitted, check_X_y
from sklearn.calibration import CalibratedClassifierCV


RANDOM_STATE = 3993
TEST_SIZE = 0.3
INTER_OPS = 0  # Independent non-blocking operations.
INTRA_OPS = 0  # Internal Matrix multiplication and reductions.

tf.config.threading.set_inter_op_parallelism_threads(INTER_OPS)
tf.config.threading.set_intra_op_parallelism_threads(INTRA_OPS)
tf.device('/cpu:0')

np.random.seed(seed=RANDOM_STATE)
tf.random.set_seed(seed=RANDOM_STATE)
random.seed(RANDOM_STATE)

2023-10-27 11:18:07.956951: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/nicholas/.conda/envs/optunaEnv2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-10-27 11:18:11.572707: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [2]:
DATASET = "adult"
DROP_DATA_PERC = 0.0

path = "./dataset/" + DATASET + ".csv"

df = pd.read_csv(path)
NUM_CLASSES = np.unique(df['class']).size
scale = sklearn.preprocessing.StandardScaler()

X, y = df.drop(columns='class').values, df['class'].values

print(X.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
)

if DROP_DATA_PERC > 0.0:
    print(X_train.shape)

    X_train, _, y_train, _ = train_test_split(
        X_train, y_train, test_size=DROP_DATA_PERC, random_state=RANDOM_STATE
    )

print(X_train.shape)

X_train = scale.fit_transform(X_train)
X_test = scale.transform(X_test)

(48842, 13)
(34189, 13)


In [3]:
def create_model():

    n_hidden = best_params["n_hidden"]
    hidden_units = best_params["hidden_units"]
    learning_rate = best_params["learning_rate"]
    dropout = best_params["dropout"]
    hidden_activation = best_params["hidden_activation"]
    batch_norm = best_params["batch_norm"]
    activity_regularizer = best_params["activity_regularizer"]

    kernel_initializer = {
        "relu": "he_uniform",
        "selu": "lecun_normal",
        "elu": "he_uniform",
        "swish": "he_uniform",
    }.get(hidden_activation, "glorot_uniform")


    model = Sequential()
    
    model.add(
        Input(
            shape=X_train.shape[1:],
        )
    )
    
    for i in range(n_hidden):
        model.add(
            Dense(
                units=hidden_units,
                activation=hidden_activation,
                activity_regularizer=tf.keras.regularizers.L2(1e-5) if activity_regularizer else None,
                kernel_initializer=kernel_initializer,
            )
        )
        model.add(
            Dropout(dropout)
        )

        if batch_norm:
            model.add(tf.keras.layers.BatchNormalization())

        hidden_units = int(hidden_units / 2)

        
    if NUM_CLASSES > 2:
        model.add(
            Dense(
                NUM_CLASSES,
                activation="softmax"
            )
        )

        model.compile(
            loss='sparse_categorical_crossentropy',
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            metrics='accuracy',
        )
    else:
        model.add(
            Dense(
                1,
                activation="sigmoid"
            )
        )

        model.compile(
            loss='binary_crossentropy',
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            metrics='accuracy',
        )

    model.load_weights(init_weights_pathname)
    
    return model

In [4]:
def multi_stratified_kfold(X_train, y_train):
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=RANDOM_STATE)

    folds = list()
    for train_index, val_index in skf.split(X_train, y_train):
        # Get labels
        y_train_fold = y_train[train_index]
        y_val_fold = y_train[val_index]
        
        # Append folds
        folds.append(
            (
                X_train[train_index],
                X_train[val_index],
                y_train_fold,
                y_val_fold,
            ),
        )

    return folds

In [5]:
storage_name = JournalStorage(JournalFileStorage("./optuna.log"))

study_name = DATASET + "-study"

study = optuna.load_study(storage=storage_name, study_name=study_name)

best_params = study.best_params

batch_size = best_params.pop("batch_size")

init_weights_pathname = "./weights/" + DATASET + "/init_weights_" + str(study.best_trial.number)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, verbose=1, mode="min", restore_best_weights=True)

/tmp/ipykernel_216427/2327146705.py:1: ExperimentalWarning: JournalStorage is experimental (supported from v3.1.0). The interface can change in the future.
  storage_name = JournalStorage(JournalFileStorage("./optuna.log"))


In [6]:
base_estimator = KerasClassifier(model=create_model, epochs=100, batch_size=batch_size, validation_split=0.2, callbacks=[callback], verbose=1)


print(base_estimator.model().summary())

base_estimator.fit(
    X_train,
    y_train,
    verbose=1
)

prediction_base = base_estimator.predict(X_test, verbose=0)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 211)               2954      
                                                                 
 dropout (Dropout)           (None, 211)               0         
                                                                 
 batch_normalization (BatchN  (None, 211)              844       
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 105)               22260     
                                                                 
 dropout_1 (Dropout)         (None, 105)               0         
                                                                 
 batch_normalization_1 (Batc  (None, 105)              420       
 hNormalization)                                        

In [9]:
print("Score Train: " + str(base_estimator.score(X_train, y_train)))
print("Score Test: " + str(base_estimator.score(X_test, y_test)))


713/713 [==============================] - 3s 4ms/step
Score Train: 0.859633215361666
306/306 [==============================] - 1s 3ms/step
Score Test: 0.8547055210537091
